<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/6/6f/Dauphine_logo_2019_-_Bleu.png" style="width: 600px;"/> 
</center>  

<div align="center"><span style="font-family:Arial Black;font-size:25px;color:darkblue">Gestion des risques - M1 Economie et Finance </span></div>

<div align="center"><span style="font-family:Arial Black;font-size:20px;color:darkblue">Projet final 2022 - 2023</span></div>

<div align="center"><span style="font-family:Arial Black;font-size:14px;color:darkblue">Par Benkaddour Joudy, BOUSSAID Nassim et MOUROUGAYA Kartty sous la direction de M. Kayo de Kayo</span></div>

# Partie C - Application à une position short sur un call européen

<u>**1. VaR théorique :**</u>

$L$ représente la variation du prix d'une option entre le temps $t$ et le temps $t+δt$. En utilisant la formule $L = C_{t+δt} − C_{t}$, nous pouvons exprimer cette variation comme la différence entre le prix de l'option en $t+δt$ et le prix de l'option en $t$. Etant donné que $δt>0$, $C_{t+δt}$ est le prix d'une option pour une échéance plus tardive que $C_{t}$. Or, nous savons que le payoff d'un call est $Max(S_{t}-K;0)$ donc plus $S_{t+δt}$ est grand et plus la différence $C_{t+δt} − C_{t}$ sera grande et par conséquent $L$ sera grand. Ainsi on peut en déduire que $L$ est une fonction croissante de $S_{t+δt}$.

In [1]:
from math import log, exp, sqrt
from scipy.stats import norm
import numpy as np

In [2]:
# Initialisation des variables
S = 917.8
K = 925
T = 51
delta_T = 14
r = 0.000045205
vol = 0.015
S_δt = S*exp((r-vol**2/2)*delta_T + vol*sqrt(delta_T)* norm.ppf(0.99))

In [3]:
# Fonction calculant d1
def d1(S,K,r,vol,T):
    vol_T = vol*sqrt(T)
    d1 = (log(S/K)+(r+vol**2/2)*T) / vol_T
    return d1

In [4]:
# Fonction calculant d2
def d2(S,K,r,vol,T):
    vol_T = vol*sqrt(T)
    d2 = d1(S,K,r,vol,T) - vol_T
    return d2

In [5]:
# Classe calculant les greeks du call avec des formules fermées
class call:
    def __init__(self,S,K,r,vol,T):
        self.S = S
        self.K = K
        self.r = r
        self.vol = vol
        self.vol_T = vol*sqrt(T)
        self.T = T
        self.d1 = d1(S,K,r,vol,T)
        self.d2 = d2(S,K,r,vol,T)
    
    def price(self):
        price = self.S*norm.cdf(self.d1) - self.K*exp(-self.r*(self.T))*norm.cdf(self.d2)
        return price
    
    def delta(self):
        delta = norm.cdf(self.d1)
        return delta
    
    def gamma(self):
        gamma = norm.pdf(self.d1)/(self.S*self.vol_T)
        return gamma
    
    def speed(self):
        speed = -norm.pdf(self.d1)/(self.S**2*(self.vol_T))*((self.d1/self.vol_T)+1)
        return speed

In [6]:
# Classe calculant les greeks du put avec des formules fermées
class put:
    def __init__(self,S,K,r,vol,T):
        self.S = S
        self.K = K
        self.r = r
        self.vol = vol
        self.vol_T = vol*sqrt(T)
        self.T = T
        self.d1 = d1(S,K,r,vol,T)
        self.d2 = d2(S,K,r,vol,T)
    
    def price(self):
        price = self.K*exp(-self.r*(self.T))*norm.cdf(-self.d2) - self.S*norm.cdf(-self.d1)
        return price
    
    def delta(self):
        delta = -norm.cdf(-self.d1)
        return delta
    
    def gamma(self):
        gamma = norm.pdf(self.d1)/(self.S*self.vol_T)
        return gamma
    
    def speed(self):
        speed = -norm.pdf(self.d1)/(self.S**2*(self.vol_T))*((self.d1/self.vol_T)+1)
        return speed

In [7]:
# Calcul de la VaR Théorique
VaR_Theorique = call(S_δt,K,r,vol,T-delta_T).price() - call(S,K,r,vol,T).price()
print(f"La VaR théorique est : {VaR_Theorique}")

La VaR théorique est : 88.17667789647226


In [8]:
# Affichage des résultats des greeks pour le call et le put
print(f"Le prix du call en T est : {call(S,K,r,vol,T).price()}")
print(f"Le prix du put en T est : {put(S,K,r,vol,T).price()}")
print(f"Le prix du call en T-delta_t est : {call(S_δt,K,r,vol,T-delta_T).price()}")
print(f"Le prix du put en T-delta_t est : {put(S_δt,K,r,vol,T-delta_T).price()}\n")

print(f"Le delta du call en T est : {call(S,K,r,vol,T).delta()}")
print(f"Le delta du put en T est : {put(S,K,r,vol,T).delta()}")
print(f"Le delta du call en T-delta_t est : {call(S_δt,K,r,vol,T-delta_T).delta()}")
print(f"Le delta du put en T-delta_t est : {put(S_δt,K,r,vol,T-delta_T).delta()}\n")

print(f"Le gamma du call en T est : {call(S,K,r,vol,T).gamma()}")
print(f"Le gamma du put en T est : {put(S,K,r,vol,T).gamma()}")
print(f"Le gamma du call en T-delta_t est : {call(S_δt,K,r,vol,T-delta_T).gamma()}")
print(f"Le gamma du put en T-delta_t est : {put(S_δt,K,r,vol,T-delta_T).gamma()}\n")

print(f"Le speed du call en T est : {call(S,K,r,vol,T).speed()}")
print(f"Le speed du put en T est : {put(S,K,r,vol,T).speed()}")
print(f"Le speed du call en T-delta_t est : {call(S_δt,K,r,vol,T-delta_T).speed()}")
print(f"Le speed du put en T-delta_t est : {put(S_δt,K,r,vol,T-delta_T).speed()}\n")

Le prix du call en T est : 36.828927782947744
Le prix du put en T est : 41.89883826418736
Le prix du call en T-delta_t est : 125.00560567942
Le prix du put en T-delta_t est : 3.6361825154471603

Le delta du call en T est : 0.5008517977401307
Le delta du put en T est : -0.49914820225986933
Le delta du call en T-delta_t est : 0.9190900547789507
Le delta du put en T-delta_t est : -0.08090994522104927

Le gamma du call en T est : 0.0040577442912202175
Le gamma du put en T est : 0.0040577442912202175
Le gamma du call en T-delta_t est : 0.0015728531591393791
Le gamma du put en T-delta_t est : 0.0015728531591393791

Le speed du call en T est : -4.509286512781038e-06
Le speed du put en T est : -4.509286512781038e-06
Le speed du call en T-delta_t est : -2.458685710137681e-05
Le speed du put en T-delta_t est : -2.458685710137681e-05



<u>**2. VaR Monte-Carlo :**</u>

In [9]:
# Fonction calculant la VaR Monte-Carlo pour le call
def Monte_Carlo_Call(S,K,r,vol,T,Nb_simulations):
    S_price = S * np.exp((r - vol**2/2)*T + vol*np.sqrt(T)*norm.rvs(size = Nb_simulations))
    Call_price = np.exp(-r*T) *np.maximum(S_price-K,0)
    Call_price_moyen = np.mean(Call_price)
    return Call_price_moyen

In [10]:
# Fonction calculant la VaR Monte-Carlo pour le put
def Monte_Carlo_Put(S,K,r,vol,T,Nb_simulations):
    S_price = S * np.exp((r - vol**2/2)*T + vol*np.sqrt(T)*norm.rvs(size = Nb_simulations))
    Put_price = np.exp(-r*T) *np.maximum(K-S_price,0)
    Put_price_moyen = np.mean(Put_price)
    return Put_price_moyen

In [11]:
# Enregistrement du nombre de simulations
Nb_simulations = 10000000

In [12]:
# Calcul de la VaR Monte-Carlo
VaR_Monte_Carlo = Monte_Carlo_Call(S_δt,K,r,vol,T-delta_T,Nb_simulations) - Monte_Carlo_Call(S,K,r,vol,T,Nb_simulations)
print(f"La VaR Monte-Carlo est : {VaR_Monte_Carlo}")
Erreur_Relative = abs((VaR_Monte_Carlo - VaR_Theorique)/VaR_Theorique)*100
print(f"L'erreur relative à la VaR théorique est : {Erreur_Relative}%")

La VaR Monte-Carlo est : 88.17768663618483
L'erreur relative à la VaR théorique est : 0.0011439983186470014%


<u>**3. VaR Cornish-Fischer :**</u>

<center>
    <img src="https://image.noelshack.com/fichiers/2023/10/4/1678317331-screenshot-59.png" style="width: 600px;"/> 
    <img src="https://www.zupimages.net/up/23/10/bh79.png" style="width: 600px;"/> 
</center>

In [13]:
# Fonction calculant la VaR Cornish-Fischer
def Cornish_Fischer_VaR(S,K,r,vol,T,option = "call",position="short",gamma_factor=1, speed_factor=1,seuil=0.01):
    # Vérification de la position
    if position == "short":
        facteur = -1
    elif position == "long":
        facteur = 1
    # Vérification de la nature de l'option
    if option == "call":
        Delta = facteur * call(S,K,r,vol,T).delta()
        Gamma = facteur * gamma_factor * call(S,K,r,vol,T).gamma()
        Speed = facteur * speed_factor * call(S,K,r,vol,T).speed()
    elif option == "put":
        Delta = facteur * put(S,K,r,vol,T).delta()
        Gamma = facteur * gamma_factor * put(S,K,r,vol,T).gamma()
        Speed = facteur * speed_factor * put(S,K,r,vol,T).speed()

    a = 1/6*Speed*S**3
    b = 1/2*Gamma*S**2
    c = Delta*S
    vol_T = vol * sqrt(delta_T)
    
    # Calcul des moments
    E_δP = b*vol_T**2
    E_δP_2 = a**2*15*vol_T**6 + (b**2+2*a*c)*3*vol_T**4 + c**2*1*vol_T**2
    E_δP_3 = 315*b*a**2*vol_T**8 + 15*(b**3 + 6*a*b*c)*vol_T**6 + 9*b*c**2*vol_T**4
    E_δP_4 = 10395*a**4*vol_T**12+945*(6*a**2*b**2+4*c*a**3)*vol_T**10+105*(b**4+12*a*c*b**2+6*a**2*c**2)*vol_T**8+15*(6*b**2*c**2+4*a*c**3)*vol_T**6+3*c**4*vol_T**4
    
    σ_δP_2 = E_δP_2-E_δP**2
    σ_δP = sqrt(σ_δP_2)
    
    S_δP = (E_δP_3-3*E_δP_2*E_δP+2*E_δP**3)/(σ_δP**3)
    K_δP = (E_δP_4-4*E_δP_3*E_δP+6*E_δP_2*E_δP**2-3*E_δP**4)/(σ_δP**4) - 3
    
    dic_z_alpha = {0.05: 1.64, 0.025: 1.96, 0.01: 2.33, 0.005: 2.58, 0.001:3.09}
    z_alpha = dic_z_alpha[seuil]

    E_L = -E_δP
    σ_L = σ_δP
    S_L = -S_δP
    K_L = K_δP
    
    # Test de Diddier-Maillard
    Condition_1 = abs(S_L)<=6*(sqrt(2)-1) or abs(S_L)>=6*(sqrt(2)+1)
    if Condition_1:
        s = S_L/6
        k = K_L/24
        k_1 = (1+11*s**2-sqrt(s**4-6*s**2+1))/6
        k_2 = (1+11*s**2+sqrt(s**4-6*s**2+1))/6
        Condition_2 = k_1 <= k <= k_2
        if Condition_2:
            z_tilde = z_alpha + (z_alpha**2-1)*S_L/6 + (z_alpha**3-3*z_alpha)*K_L/24 - (2*z_alpha**3-5*z_alpha)*S_L**2/36
            print("Les conditions de validité de Diddier Maillard sont vérifiées")
        else:
            z_tilde = z_alpha + (z_alpha**2-1)*S_L/6
            print("Les conditions de validité de Diddier Maillard ne sont pas vérifiées")
    else:
        z_tilde = z_alpha + (z_alpha**2-1)*S_L/6
        print("Les conditions de validité de Diddier Maillard ne sont pas vérifiées")
    VaR_Cornish_Fischer = E_L+σ_L*z_tilde
    return VaR_Cornish_Fischer

In [14]:
# Calcul de la VaR Cornish Fischer
VaR_Cornish_Fischer = Cornish_Fischer_VaR(S,K,r,vol,T,position="short",gamma_factor=0, speed_factor=0,seuil=0.01)
Erreur_Relative = abs((VaR_Cornish_Fischer - VaR_Theorique)/VaR_Theorique)*100
print(f"La VaR Cornish-Fischer avec Delta uniquement est : {VaR_Cornish_Fischer}")
print(f"L'erreur relative à la VaR théorique est : {Erreur_Relative}%\n")

VaR_Cornish_Fischer = Cornish_Fischer_VaR(S,K,r,vol,T,position="short",gamma_factor=1, speed_factor=0,seuil=0.01)
Erreur_Relative = abs((VaR_Cornish_Fischer - VaR_Theorique)/VaR_Theorique)*100
print(f"La VaR Cornish-Fischer avec Delta-Gamma est : {VaR_Cornish_Fischer}")
print(f"L'erreur relative à la VaR théorique est : {Erreur_Relative}%\n")

VaR_Cornish_Fischer = Cornish_Fischer_VaR(S,K,r,vol,T,position="short",gamma_factor=1, speed_factor=1,seuil=0.01)
Erreur_Relative = abs((VaR_Cornish_Fischer - VaR_Theorique)/VaR_Theorique)*100
print(f"La VaR Cornish-Fischer avec Delta-Gamma-Speed est : {VaR_Cornish_Fischer}")
print(f"L'erreur relative à la VaR théorique est : {Erreur_Relative}%\n")

Les conditions de validité de Diddier Maillard sont vérifiées
La VaR Cornish-Fischer avec Delta uniquement est : 60.11301187873832
L'erreur relative à la VaR théorique est : 31.8266311310609%

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta-Gamma est : 91.26536476335828
L'erreur relative à la VaR théorique est : 3.5028387784266855%

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta-Gamma-Speed est : 90.04068533545545
L'erreur relative à la VaR théorique est : 2.1139460948752395%



# Partie D - Application à une position short sur un straddle

In [15]:
# Initialisation des variables
S = 100
K = 100
T = 28
delta_T = 14
r = 0.02/252
vol = 0.015

In [16]:
# Initialisation du portefeuille
Put_T = put(S,K,r,vol,T).price()
Call_T = call(S,K,r,vol,T).price()
Portfolio_T = -(Call_T + Put_T)

In [17]:
# Calcul de la VaR Monte-Carlo
Nb_simulations = 100000
S_δt = S * np.exp((r - vol**2/2)*(T-delta_T) + vol*np.sqrt(T-delta_T)*norm.rvs(size = Nb_simulations))
Portfolio_δt = np.empty(Nb_simulations, dtype=float)
for i in range(1,Nb_simulations):
    Portfolio_δt[i] = -(call(S_δt[i],K,r,vol,T-delta_T).price() + put(S_δt[i],K,r,vol,T-delta_T).price())
L = Portfolio_T - Portfolio_δt
VaR_Monte_Carlo = (np.quantile(L,0.99))
print(f"La VaR Monte-Carlo est : {VaR_Monte_Carlo}")

La VaR Monte-Carlo est : 8.33027096094263


In [18]:
# Fonction calculant la VaR Cornish-Fischer
def Cornish_Fischer_VaR_Straddle(S,K,r,vol,T,position="short", gamma_factor=1, speed_factor=1, seuil=0.01):
    # Vérification de la position
    if position == "short":
        facteur = -1
    elif position == "long":
        facteur = 1
    
    Delta = facteur * (call(S,K,r,vol,T).delta() + put(S,K,r,vol,T).delta())
    Gamma = facteur * gamma_factor * (call(S,K,r,vol,T).gamma() + put(S,K,r,vol,T).gamma())
    Speed = facteur * speed_factor * (call(S,K,r,vol,T).speed() + put(S,K,r,vol,T).speed())

    a = 1/6*Speed*S**3
    b = 1/2*Gamma*S**2
    c = Delta*S
    vol_T = vol * sqrt(delta_T)
    # Calcul des moments
    E_δP = b*vol_T**2
    E_δP_2 = a**2*15*vol_T**6 + (b**2+2*a*c)*3*vol_T**4 + c**2*1*vol_T**2
    E_δP_3 = 315*b*a**2*vol_T**8 + 15*(b**3 + 6*a*b*c)*vol_T**6 + 9*b*c**2*vol_T**4
    E_δP_4 = 10395*a**4*vol_T**12+945*(6*a**2*b**2+4*c*a**3)*vol_T**10+105*(b**4+12*a*c*b**2+6*a**2*c**2)*vol_T**8+15*(6*b**2*c**2+4*a*c**3)*vol_T**6+3*c**4*vol_T**4
    
    σ_δP_2 = E_δP_2-E_δP**2
    σ_δP = sqrt(σ_δP_2)
    
    S_δP = (E_δP_3-3*E_δP_2*E_δP+2*E_δP**3)/(σ_δP**3)
    K_δP = (E_δP_4-4*E_δP_3*E_δP+6*E_δP_2*E_δP**2-3*E_δP**4)/(σ_δP**4) - 3
    
    dic_z_alpha = {0.05: 1.64, 0.025: 1.96, 0.01: 2.33, 0.005: 2.58, 0.001:3.09}
    z_alpha = dic_z_alpha[seuil]

    E_L = -E_δP
    σ_L = σ_δP
    S_L = -S_δP
    K_L = K_δP
    # Test de Diddier-Maillard
    Condition_1 = abs(S_L)<=6*(sqrt(2)-1) or abs(S_L)>=6*(sqrt(2)+1)
    if Condition_1:
        s = S_L/6
        k = K_L/24
        k_1 = (1+11*s**2-sqrt(s**4-6*s**2+1))/6
        k_2 = (1+11*s**2+sqrt(s**4-6*s**2+1))/6
        Condition_2 = k_1 <= k <= k_2
        if Condition_2:
            z_tilde = z_alpha + (z_alpha**2-1)*S_L/6 + (z_alpha**3-3*z_alpha)*K_L/24 - (2*z_alpha**3-5*z_alpha)*S_L**2/36
            print("Les conditions de validité de Diddier Maillard sont vérifiées")
        else:
            z_tilde = z_alpha + (z_alpha**2-1)*S_L/6
            print("Les conditions de validité de Diddier Maillard ne sont pas vérifiées")
    else:
        z_tilde = z_alpha + (z_alpha**2-1)*S_L/6
        print("Les conditions de validité de Diddier Maillard ne sont pas vérifiées")
    VaR_Cornish_Fischer = E_L+σ_L*z_tilde
    return VaR_Cornish_Fischer

In [19]:
# Calcul de la VaR Cornish Fischer
VaR_Cornish_Fischer = Cornish_Fischer_VaR_Straddle(S,K,r,vol,T,position="short",gamma_factor=0, speed_factor=0,seuil=0.01)
print(f"La VaR Cornish-Fischer avec Delta uniquement est : {VaR_Cornish_Fischer}\n")

VaR_Cornish_Fischer = Cornish_Fischer_VaR_Straddle(S,K,r,vol,T,position="short",gamma_factor=1, speed_factor=0,seuil=0.01)
print(f"La VaR Cornish-Fischer avec Delta-Gamma est : {VaR_Cornish_Fischer}\n")

VaR_Cornish_Fischer = Cornish_Fischer_VaR_Straddle(S,K,r,vol,T,position="short",gamma_factor=1, speed_factor=1,seuil=0.01)
print(f"La VaR Cornish-Fischer avec Delta-Gamma-Speed est : {VaR_Cornish_Fischer}\n")

Les conditions de validité de Diddier Maillard sont vérifiées
La VaR Cornish-Fischer avec Delta uniquement est : 0.7056729424758763

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta-Gamma est : 11.53842476597822

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta-Gamma-Speed est : 11.472031339338645



# Partie E - Application à un portefeuille quelconque d'options

In [20]:
# Initialisation des variables
S = 100
K_Put = 97
K_Call_1 = 97
K_Call_2 = 105
Q_Put = -1
Q_Call_1 = -1.5
Q_Call_2 = 2.5
T = 28
delta_T = 14
r = 0.02/252
vol = 0.015

In [21]:
# Initialisation du portefeuille
Put_T = put(S,K_Put,r,vol,T).price()
Call_1_T = call(S,K_Call_1,r,vol,T).price()
Call_2_T = call(S,K_Call_2,r,vol,T).price()
Portfolio_T = (Q_Put*Put_T + Q_Call_1*Call_1_T + Q_Call_2*Call_2_T)

In [22]:
# Calcul de la VaR Monte-Carlo
Nb_simulations = 100000
S_δt = S * np.exp((r - vol**2/2)*(T-delta_T) + vol*np.sqrt(T-delta_T)*norm.rvs(size = Nb_simulations))
Portfolio_δt = np.empty(Nb_simulations, dtype=float)
for i in range(1,Nb_simulations):
    Portfolio_δt[i] = (Q_Put*put(S_δt[i],K_Put,r,vol,T-delta_T).price() + Q_Call_1*call(S_δt[i],K_Call_1,r,vol,T-delta_T).price() + Q_Call_2*call(S_δt[i],K_Call_2,r,vol,T-delta_T).price())
L = Portfolio_T - Portfolio_δt
VaR_Monte_Carlo = (np.quantile(L,0.99))
print(f"La VaR Monte-Carlo est : {VaR_Monte_Carlo}")

La VaR Monte-Carlo est : 3.678445855132349


In [23]:
# Fonction calculant la VaR Cornish-Fischer
def Cornish_Fischer_VaR_Portfolio(S,r,vol,T,gamma_factor=1, speed_factor=1, seuil=0.01):
    # Vérification de la position
    Delta = - put(S,K_Put,r,vol,T).delta() - 1.5*call(S,K_Call_1,r,vol,T).delta() + 2.5*call(S,K_Call_2,r,vol,T).delta()
    Gamma = gamma_factor*(- put(S,K_Put,r,vol,T).gamma() - 1.5*call(S,K_Call_1,r,vol,T).gamma() + 2.5*call(S,K_Call_2,r,vol,T).gamma())
    Speed = speed_factor*(- put(S,K_Put,r,vol,T).speed() - 1.5*call(S,K_Call_1,r,vol,T).speed() + 2.5*call(S,K_Call_2,r,vol,T).speed())

    a = 1/6*Speed*S**3
    b = 1/2*Gamma*S**2
    c = Delta*S
    vol_T = vol * sqrt(delta_T)
    # Calcul des moments
    E_δP = b*vol_T**2
    E_δP_2 = a**2*15*vol_T**6 + (b**2+2*a*c)*3*vol_T**4 + c**2*1*vol_T**2
    E_δP_3 = 315*b*a**2*vol_T**8 + 15*(b**3 + 6*a*b*c)*vol_T**6 + 9*b*c**2*vol_T**4
    E_δP_4 = 10395*a**4*vol_T**12+945*(6*a**2*b**2+4*c*a**3)*vol_T**10+105*(b**4+12*a*c*b**2+6*a**2*c**2)*vol_T**8+15*(6*b**2*c**2+4*a*c**3)*vol_T**6+3*c**4*vol_T**4

    σ_δP_2 = E_δP_2-E_δP**2
    σ_δP = sqrt(σ_δP_2)

    S_δP = (E_δP_3-3*E_δP_2*E_δP+2*E_δP**3)/(σ_δP**3)
    K_δP = (E_δP_4-4*E_δP_3*E_δP+6*E_δP_2*E_δP**2-3*E_δP**4)/(σ_δP**4) - 3

    dic_z_alpha = {0.05: 1.64, 0.025: 1.96, 0.01: 2.33, 0.005: 2.58, 0.001:3.09}
    z_alpha = dic_z_alpha[seuil]

    E_L = -E_δP
    σ_L = σ_δP
    S_L = -S_δP
    K_L = K_δP
    # Test de Diddier-Maillard
    Condition_1 = abs(S_L)<=6*(sqrt(2)-1) or abs(S_L)>=6*(sqrt(2)+1)
    if Condition_1:
        s = S_L/6
        k = K_L/24
        k_1 = (1+11*s**2-sqrt(s**4-6*s**2+1))/6
        k_2 = (1+11*s**2+sqrt(s**4-6*s**2+1))/6
        Condition_2 = k_1 <= k <= k_2
        if Condition_2:
            z_tilde = z_alpha + (z_alpha**2-1)*S_L/6 + (z_alpha**3-3*z_alpha)*K_L/24 - (2*z_alpha**3-5*z_alpha)*S_L**2/36
            print("Les conditions de validité de Diddier Maillard sont vérifiées")
        else:
            z_tilde = z_alpha + (z_alpha**2-1)*S_L/6
            print("Les conditions de validité de Diddier Maillard ne sont pas vérifiées")
    else:
        z_tilde = z_alpha + (z_alpha**2-1)*S_L/6
        print("Les conditions de validité de Diddier Maillard ne sont pas vérifiées")
    VaR_Cornish_Fischer = E_L+σ_L*z_tilde
    return VaR_Cornish_Fischer

In [24]:
# Calcul de la VaR Cornish Fischer
VaR_Cornish_Fischer = Cornish_Fischer_VaR_Portfolio(S,r,vol,T,gamma_factor=0, speed_factor=0, seuil=0.01)
print(f"La VaR Cornish-Fischer avec Delta uniquement est : {VaR_Cornish_Fischer}\n")
VaR_Cornish_Fischer = Cornish_Fischer_VaR_Portfolio(S,r,vol,T,gamma_factor=1, speed_factor=0, seuil=0.01)
print(f"La VaR Cornish-Fischer avec Delta-Gamma est : {VaR_Cornish_Fischer}\n")
VaR_Cornish_Fischer = Cornish_Fischer_VaR_Portfolio(S,r,vol,T,gamma_factor=1, speed_factor=1, seuil=0.01)
print(f"La VaR Cornish-Fischer avec Delta-Gamma-Speed est : {VaR_Cornish_Fischer}\n")

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta uniquement est : 0.5892170274219752

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta-Gamma est : 1.0802064828850058

Les conditions de validité de Diddier Maillard ne sont pas vérifiées
La VaR Cornish-Fischer avec Delta-Gamma-Speed est : 5.295487463043295

